In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import matplotlib.pyplot as plt

In [3]:
torch.manual_seed(42)

In [4]:
df = pd.read_csv("fmnist_small.csv")
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [5]:
#train test split
x = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [6]:
x_train,x_test,y_train,y_test = train_test_split(x,y , test_size=0.2 , random_state=42)

In [7]:
#scaling the features, basically we are normalizing the values to an smaller intervals (0-1)
x_train = x_train/255.0
x_test = x_test/255.0

In [8]:
#creating custom dataset
class CustomDataset(Dataset):

    def __init__(self,features,labels):
        
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels,dtype=torch.long)
    
    def __len__(self):
        return len(self.features)

    def __getitem__(self,index):
        return self.features[index],self.labels[index]

In [9]:
#create train_dataset object
train_dataset = CustomDataset(x_train,y_train)

In [10]:
len(train_dataset)

4800

In [11]:
#create train_dataset object
test_dataset = CustomDataset(x_test,y_test)

In [12]:
len(test_dataset)

1200

In [13]:
#creating train and test loader
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [14]:
#define nn class
class MyNN(nn.Module):
    
    def __init__(self,num_features):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(num_features,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,10)
        )
    def forward(self,x):
        return self.model(x)

In [15]:
#set learning rate and epochs
learning_rate = 0.1
epochs = 100

In [16]:
#instantiate the model
model = MyNN(x_train.shape[1])

#loss function
criterion = nn.CrossEntropyLoss()

#optimizer
optimizer = optim.SGD(model.parameters(),lr =learning_rate)

In [18]:
#training loop
for epoch in range(epochs):
    for batch_features, batch_labels in train_loader:

        #forward pass
        output= model(batch_features)
        #loss calculation
        loss =criterion(output,batch_labels)
        #backpropagation
        optimizer.zero_grad()
        loss.backward()
        #update the parameters
        optimizer.step()